### Putting all 4 dragon balls together, and summon the divine dragon...

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.kernel_ridge import KernelRidge

import time

from catboost import CatBoostRegressor

import re

from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVR

# We will write customized regressor classes which inherits the following base classes from sklearn.
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

import pickle

from dictionaries import *;
from RegressorEncapsulation import *;

pd.set_option('display.max_columns', 500)

#### Preparation

In [2]:
housing_coords = pd.read_csv('../data/ames_housing_price_data_v6.csv', index_col = 0);
if "SalePrice.1" in housing_coords.columns:
    housing_coords.drop("SalePrice.1", axis = 1, inplace = True);

y = housing_coords["SalePrice"]
# y_std = svr_price_scaler.fit_transform(np.array(np.log10(y)).reshape(-1,1));
front_end = housing_coords.copy();
front_end.reset_index(inplace = True)

In [32]:
# Loading files and steal the parameters.
cbl = CatBoostRegressor(subsample= 0.85, depth= 2, random_seed= 0, learning_rate= 0.04, iterations= 4000, verbose=False);
lm = Lasso(alpha=1e-06, copy_X=True, fit_intercept= True, max_iter= 1000, normalize= True, positive= False, precompute= False, selection= 'cyclic', tol= 0.001);
svrg = SVR(C= 6000, epsilon = 0.1, gamma = 6e-5, max_iter=-1, shrinking=True);

# svrl = SVR(C= 100, cache_size=200, coef0 = 0.0, epsilon = 0.1, kernel = "linear", max_iter=-1, shrinking=True);
svrl = KernelRidge(alpha=0.005, coef0 = 0.0, kernel = "linear");

In [ ]:
# with open('../Matt/linearmodel.pickle', mode = 'rb') as file:
#     lm = pickle.load(file);

# with open('../Matt/SVR_model.pickle', mode = 'rb') as file:
#     svrl = pickle.load(file);

# with open('SVR_model_g.pickle', mode = 'rb') as file:
#     svrg = pickle.load(file);


In [23]:
svrl = SVR(C= 100, cache_size=200, coef0 = 0.0, epsilon = 0.1, kernel = "linear", max_iter=-1, shrinking=True);
# svrl = KernelRidge(alpha=0.005, coef0 = 0.0, kernel = "linear");

In [24]:
# from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
class EncapsulatedModel(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, method="cat", instance = CatBoostRegressor()):
        self.method = method;
        self.instance = instance;
        self.x_scaler = StandardScaler();
        self.y_scaler = StandardScaler();
        self.fitted = False;
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        back_end = front_to_back(X, self.method, self.x_scaler, True);
        y_proc = predictor_processing(y, self.method, self.y_scaler);
        self.instance.fit(back_end, y_proc);
        self.fitted = True;
        return self;
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        return predict_from_front(X, self.method, self.instance, self.x_scaler, self.y_scaler);


In [69]:
[1] + [0]*3

[1, 0, 0, 0]

In [56]:
a = np.array((1,2,3))
b = np.array((2,3,4))
c = np.column_stack((a,b))

In [68]:
c
np.matmul(c, np.array([1,3]))

array([ 7, 11, 15])

In [90]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models, weight=None):
        self.models = models
        if not weight:
            self.weight= [1] + [0]*(len(models)-1);
        else: self.weight = weight;
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)
        return self
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.matmul(predictions, np.array(self.weight))
        # return np.mean(predictions, axis=1)

In [25]:
x = EncapsulatedModel("svrl", svrl);

In [26]:
x.fit(front_end, y)

EncapsulatedModel(instance=SVR(C=100, kernel='linear'), method='svrl')

In [27]:
y_pred = x.predict(front_end)

In [28]:
x.score(front_end, y) # Kernel ridge: 0.9391

0.9364478042572866

In [29]:
X_train, X_test, y_train, y_test = train_test_split(front_end, y, test_size=0.3, random_state=1);

In [30]:
x.fit(X_train, y_train);
x.score(X_test, y_test) # Kernel ridge: 0.9319

0.9322521550355579

In [73]:
averaged_models = AveragingModels(models = (EncapsulatedModel("cat", cbl), EncapsulatedModel("lm", lm),\
                                           EncapsulatedModel("svrl", svrl), EncapsulatedModel("svrg", svrg)),\
                                 weight = [1/4, 1/4, 1/4, 1/4]);
averaged_models.fit(front_end, y)

[0.25, 0.25, 0.25, 0.25]


AveragingModels(models=(EncapsulatedModel(instance=<catboost.core.CatBoostRegressor object at 0x16e691490>),
                        EncapsulatedModel(instance=Lasso(alpha=1e-06,
                                                         normalize=True,
                                                         tol=0.001),
                                          method='lm'),
                        EncapsulatedModel(instance=KernelRidge(alpha=0.005,
                                                               coef0=0.0),
                                          method='svrl'),
                        EncapsulatedModel(instance=SVR(C=6000, gamma=6e-05),
                                          method='svrg')),
                weight=[0.25, 0.25, 0.25, 0.25])

In [74]:
averaged_models.score(front_end, y)

0.9619382830879044

In [75]:
X_train, X_test, y_train, y_test = train_test_split(front_end, y, test_size=0.3, random_state=9);

In [76]:
%time averaged_models.fit(X_train, y_train);
%time averaged_models.score(X_test, y_test) # 0.9432789901860964

CPU times: user 9.4 s, sys: 3.24 s, total: 12.6 s
Wall time: 3.45 s
CPU times: user 1.82 s, sys: 187 ms, total: 2.01 s
Wall time: 356 ms


0.9432789901860964

### Applying GridSearchCV to find the best weight.

In [112]:
from itertools import combinations;

BINS = 10;
weight_list = [];
temp = list(range(BINS+3));
for combo in combinations(temp, 3):
    weight_list.append([combo[0]/BINS, (combo[1]-combo[0]-1)/BINS, (combo[2]-combo[1]-1)/BINS, (2+BINS-combo[2])/BINS]);


In [113]:
kfold = KFold(n_splits=5, shuffle = True);
params = {'weight': weight_list};
wavg = AveragingModels(models = (EncapsulatedModel("cat", cbl), EncapsulatedModel("lm", lm),\
                                           EncapsulatedModel("svrl", svrl), EncapsulatedModel("svrg", svrg)));
wavg_tuner = GridSearchCV(wavg, params, cv=kfold, return_train_score = True, n_jobs = -1)
%time wavg_tuner.fit(front_end, y)


[0.3, 0.0, 0.0, 0.7]
CPU times: user 24.6 s, sys: 6.8 s, total: 31.4 s
Wall time: 39min 52s


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=AveragingModels(models=(EncapsulatedModel(instance=<catboost.core.CatBoostRegressor object at 0x16e691490>),
                                               EncapsulatedModel(instance=Lasso(alpha=1e-06,
                                                                                normalize=True,
                                                                                tol=0.001),
                                                                 method='lm'),
                                               EncapsulatedModel(instance=KernelRidge(alpha=0.005,
                                                                                      coef0=0.0),
                                                                 method='svrl'),
                                               Encapsula...
                                    [0.0, 0.1, 0.1, 0.8], [0.0, 0.1, 0.2, 0.7],
                      

In [108]:
wavg_tuner.cv_results_

{'mean_fit_time': array([ 8.9808754 ,  9.14942832,  9.5172864 , ..., 11.36299691,
        11.19436383,  9.98894997]),
 'std_fit_time': array([0.08050487, 0.27250839, 0.09159309, ..., 0.08568085, 0.07453105,
        0.73717941]),
 'mean_score_time': array([0.49601946, 0.48918715, 0.50016613, ..., 0.59797268, 0.59856939,
        0.35993805]),
 'std_score_time': array([0.01003435, 0.00729602, 0.02159841, ..., 0.02015408, 0.01668473,
        0.11227581]),
 'param_weight': masked_array(data=[list([0.0, 0.0, 0.0, 1.0]),
                    list([0.0, 0.0, 0.05, 0.95]),
                    list([0.0, 0.0, 0.1, 0.9]), ...,
                    list([0.95, 0.0, 0.05, 0.0]),
                    list([0.95, 0.05, 0.0, 0.0]),
                    list([1.0, 0.0, 0.0, 0.0])],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'weight': [0.0, 0.0, 0.0, 1.0]},
  {'weight': [0.0, 0.0, 0.05, 0.95]},
  {'weight': [0.0, 0.0, 

In [114]:
wavg_tuner.cv_results_["mean_test_score"]

array([0.94919268, 0.94906974, 0.94862531, 0.94785939, 0.94677197,
       0.94536307, 0.94363268, 0.9415808 , 0.93920743, 0.93651256,
       0.93349621, 0.94909569, 0.94865167, 0.94788616, 0.94679916,
       0.94539067, 0.94366069, 0.94160921, 0.93923625, 0.9365418 ,
       0.93352586, 0.94867731, 0.94791221, 0.94682562, 0.94541753,
       0.94368796, 0.9416369 , 0.93926435, 0.93657031, 0.93355478,
       0.94793753, 0.94685135, 0.94544368, 0.94371452, 0.94166386,
       0.93929172, 0.93659809, 0.93358297, 0.94687635, 0.94546909,
       0.94374034, 0.9416901 , 0.93931837, 0.93662515, 0.93361044,
       0.94549378, 0.94376544, 0.94171561, 0.93934429, 0.93665148,
       0.93363718, 0.94378981, 0.94174039, 0.93936948, 0.93667708,
       0.93366319, 0.94176445, 0.93939395, 0.93670196, 0.93368848,
       0.93941769, 0.93672612, 0.93371305, 0.93674954, 0.93373688,
       0.93375999, 0.95011078, 0.94992723, 0.94942219, 0.94859567,
       0.94744765, 0.94597814, 0.94418714, 0.94207465, 0.93964

In [115]:
wavg_tuner.best_params_ # 'weight': [0.25, 0.0, 0.0, 0.75]

{'weight': [0.3, 0.0, 0.0, 0.7]}

In [120]:
wavg_tuner.best_index_

166

#### Polling

In [151]:
n_polls = 0;
polls = [0,0,0,0];
record = [];

In [156]:
for i in range(50):
    n_polls += 1;
    params = {"weight":[]};
    for j in range(4):
        temp = polls[:];
        temp[j] += 1;
        params["weight"].append([x/n_polls for x in temp]);
    
    
    wavg = AveragingModels(models = (EncapsulatedModel("cat", cbl), EncapsulatedModel("lm", lm),\
                                    EncapsulatedModel("svrl", svrl), EncapsulatedModel("svrg", svrg)));
    wavg_tuner = GridSearchCV(wavg, params, cv=kfold, return_train_score = True, n_jobs = -1)
    %time wavg_tuner.fit(front_end, y)
    
    polls[wavg_tuner.best_index_] += 1;
    record.append([wavg_tuner.best_params_, wavg_tuner.best_score_, wavg_tuner.cv_results_['mean_test_score']]);
    print(record[-1])


[0.304635761589404, 0.0, 0.0, 0.695364238410596]
CPU times: user 13.7 s, sys: 4.18 s, total: 17.9 s
Wall time: 33.4 s
[{'weight': [0.304635761589404, 0.0, 0.0, 0.695364238410596]}, 0.9521574894867981, array([0.95214417, 0.95214327, 0.95214299, 0.95215749])]
[0.3026315789473684, 0.0, 0.0, 0.6973684210526315]
CPU times: user 15.1 s, sys: 4.65 s, total: 19.8 s
Wall time: 36.5 s
[{'weight': [0.3026315789473684, 0.0, 0.0, 0.6973684210526315]}, 0.952178153882782, array([0.95216529, 0.95215686, 0.95215649, 0.95217815])]
[0.30718954248366015, 0.0, 0.0, 0.6928104575163399]
CPU times: user 14.1 s, sys: 4.38 s, total: 18.5 s
Wall time: 37.5 s
[{'weight': [0.30718954248366015, 0.0, 0.0, 0.6928104575163399]}, 0.9516423058296338, array([0.95164231, 0.95162198, 0.95162177, 0.95163058])]
[0.3051948051948052, 0.0, 0.0, 0.6948051948051948]
CPU times: user 13.5 s, sys: 4.69 s, total: 18.2 s
Wall time: 37.5 s
[{'weight': [0.3051948051948052, 0.0, 0.0, 0.6948051948051948]}, 0.9507350524461533, array([0.950

[0.3218390804597701, 0.0, 0.0, 0.6781609195402298]
CPU times: user 14.5 s, sys: 4.84 s, total: 19.4 s
Wall time: 39.9 s
[{'weight': [0.3218390804597701, 0.0, 0.0, 0.6781609195402298]}, 0.9518737415921297, array([0.95187374, 0.95185772, 0.95185677, 0.95186502])]
[1.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0]
[0.5, 0.0, 0.0, 0.5]
[0.0, 0.0, 0.5, 0.5]
[0.6666666666666666, 0.0, 0.0, 0.3333333333333333]
[0.3333333333333333, 0.0, 0.3333333333333333, 0.3333333333333333]
[0.25, 0.25, 0.0, 0.5]
[0.25, 0.0, 0.25, 0.5]
[0.4, 0.0, 0.0, 0.6]
[0.2, 0.0, 0.0, 0.8]
[0.16666666666666666, 0.16666666666666666, 0.0, 0.6666666666666666]
[0.16666666666666666, 0.16666666666666666, 0.0, 0.6666666666666666]
[0.16666666666666666, 0.0, 0.0, 0.8333333333333334]
[0.14285714285714285, 0.14285714285714285, 0.0, 0.7142857142857143]
[0.14285714285714285, 0.0, 0.14285714285714285, 0.7142857142857143]
[0.14285714285714285, 0.0, 0.0, 0.8571428571428571]
[0.25, 0.125, 0.0, 0.625]
[0.25, 0.0, 0.125, 0.625]
[0.25, 0.0, 0.0, 0.75

[0.32, 0.0, 0.0, 0.68]
CPU times: user 14.3 s, sys: 4.91 s, total: 19.2 s
Wall time: 40.1 s
[{'weight': [0.32, 0.0, 0.0, 0.68]}, 0.95074144480747, array([0.95072675, 0.95072658, 0.95072613, 0.95074144])]
[1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0]
[0.0, 0.5, 0.0, 0.5]
[0.0, 0.5, 0.0, 0.5]
[0.0, 0.0, 0.0, 1.0]
[0.3333333333333333, 0.3333333333333333, 0.0, 0.3333333333333333]
[0.3333333333333333, 0.0, 0.3333333333333333, 0.3333333333333333]
[0.3333333333333333, 0.0, 0.0, 0.6666666666666666]
[0.25, 0.25, 0.0, 0.5]
[0.25, 0.0, 0.25, 0.5]
[0.25, 0.0, 0.0, 0.75]
[0.4, 0.0, 0.0, 0.6]
[0.2, 0.0, 0.2, 0.6]
[0.16666666666666666, 0.16666666666666666, 0.0, 0.6666666666666666]
[0.16666666666666666, 0.0, 0.16666666666666666, 0.6666666666666666]
[0.2857142857142857, 0.0, 0.0, 0.7142857142857143]
[0.14285714285714285, 0.0, 0.14285714285714285, 0.7142857142857143]
[0.375, 0.0, 0.0, 0.625]
[0.25, 0.0, 0.125, 0.625]
[0.25, 0.0, 0.0, 0.75]
[0.2222222222222222, 0.1111111111111111, 0.0, 0

[1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0]
[0.0, 0.5, 0.0, 0.5]
[0.0, 0.0, 0.0, 1.0]
[0.6666666666666666, 0.0, 0.0, 0.3333333333333333]
[0.3333333333333333, 0.0, 0.3333333333333333, 0.3333333333333333]
[0.5, 0.0, 0.0, 0.5]
[0.25, 0.25, 0.0, 0.5]
[0.25, 0.0, 0.0, 0.75]
[0.4, 0.0, 0.0, 0.6]
[0.2, 0.2, 0.0, 0.6]
[0.2, 0.0, 0.0, 0.8]
[0.3333333333333333, 0.0, 0.0, 0.6666666666666666]
[0.16666666666666666, 0.0, 0.16666666666666666, 0.6666666666666666]
[0.14285714285714285, 0.14285714285714285, 0.0, 0.7142857142857143]
[0.14285714285714285, 0.0, 0.0, 0.8571428571428571]
[0.375, 0.0, 0.0, 0.625]
[0.25, 0.125, 0.0, 0.625]
[0.3333333333333333, 0.0, 0.0, 0.6666666666666666]
[0.2222222222222222, 0.0, 0.1111111111111111, 0.6666666666666666]
[0.3, 0.0, 0.0, 0.7]
[0.2, 0.0, 0.1, 0.7]
[0.2727272727272727, 0.0, 0.0, 0.7272727272727273]
[0.18181818181818182, 0.09090909090909091, 0.0, 0.7272727272727273]
[0.18181818181818182, 0.0, 0.0, 0.8181818181818182]
[0.3333333333333333, 0.0, 0.

[0.3181818181818182, 0.0, 0.0, 0.6818181818181818]
CPU times: user 14.7 s, sys: 4.58 s, total: 19.3 s
Wall time: 40 s
[{'weight': [0.3181818181818182, 0.0, 0.0, 0.6818181818181818]}, 0.9524153094125685, array([0.95241263, 0.95239993, 0.95239973, 0.95241531])]
[0.3163841807909605, 0.0, 0.0, 0.6836158192090396]
CPU times: user 14.8 s, sys: 4.62 s, total: 19.5 s
Wall time: 40.1 s
[{'weight': [0.3163841807909605, 0.0, 0.0, 0.6836158192090396]}, 0.9513719178231395, array([0.9513681 , 0.95135928, 0.95135838, 0.95137192])]
[0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 1.0]
[0.5, 0.0, 0.0, 0.5]
[0.0, 0.0, 0.5, 0.5]
[0.3333333333333333, 0.3333333333333333, 0.0, 0.3333333333333333]
[0.3333333333333333, 0.0, 0.0, 0.6666666666666666]
[0.5, 0.0, 0.0, 0.5]
[0.25, 0.0, 0.25, 0.5]
[0.2, 0.2, 0.0, 0.6]
[0.2, 0.0, 0.2, 0.6]
[0.3333333333333333, 0.0, 0.0, 0.6666666666666666]
[0.16666666666666666, 0.0, 0.16666666666666666, 0.6666666666666666]
[0.2857142857142857, 0.0, 0.0, 0.7142857142857143]


...

[0.31666666666666665, 0.0, 0.0, 0.6833333333333333]
CPU times: user 14.3 s, sys: 4.99 s, total: 19.3 s
Wall time: 40.2 s
[{'weight': [0.31666666666666665, 0.0, 0.0, 0.6833333333333333]}, 0.9504750166753387, array([0.95047502, 0.95046415, 0.95046332, 0.95047249])]
[0.3149171270718232, 0.0, 0.0, 0.6850828729281768]
CPU times: user 14.8 s, sys: 4.69 s, total: 19.4 s
Wall time: 40.2 s
[{'weight': [0.3149171270718232, 0.0, 0.0, 0.6850828729281768]}, 0.951958673885137, array([0.95194686, 0.95194309, 0.95194266, 0.95195867])]
[0.0, 1.0, 0.0, 0.0]
[0.0, 0.0, 1.0, 0.0]
[0.5, 0.0, 0.0, 0.5]
[0.0, 0.5, 0.0, 0.5]
[0.0, 0.0, 0.0, 1.0]
[0.6666666666666666, 0.0, 0.0, 0.3333333333333333]
[0.3333333333333333, 0.3333333333333333, 0.0, 0.3333333333333333]
[0.3333333333333333, 0.0, 0.0, 0.6666666666666666]
[0.25, 0.25, 0.0, 0.5]
[0.25, 0.0, 0.0, 0.75]
[0.4, 0.0, 0.0, 0.6]
[0.2, 0.0, 0.2, 0.6]
[0.2, 0.0, 0.0, 0.8]
[0.16666666666666666, 0.16666666666666666, 0.0, 0.6666666666666666]
[0.16666666666666666, 0.0

[0.3131868131868132, 0.0, 0.0, 0.6868131868131868]
CPU times: user 13.8 s, sys: 4.93 s, total: 18.7 s
Wall time: 40 s
[{'weight': [0.3131868131868132, 0.0, 0.0, 0.6868131868131868]}, 0.9514065165308807, array([0.95140423, 0.95138681, 0.95138658, 0.95140652])]
[0.3114754098360656, 0.0, 0.0, 0.6885245901639344]
CPU times: user 14.5 s, sys: 4.64 s, total: 19.2 s
Wall time: 40 s
[{'weight': [0.3114754098360656, 0.0, 0.0, 0.6885245901639344]}, 0.951927233379218, array([0.95192261, 0.95191605, 0.95191552, 0.95192723])]
[0.30978260869565216, 0.0, 0.0, 0.6902173913043478]
CPU times: user 14.6 s, sys: 4.6 s, total: 19.2 s
Wall time: 40.3 s
[{'weight': [0.30978260869565216, 0.0, 0.0, 0.6902173913043478]}, 0.9507567239546155, array([0.95073561, 0.95074407, 0.95074407, 0.95075672])]
[0.31351351351351353, 0.0, 0.0, 0.6864864864864865]
CPU times: user 14.6 s, sys: 4.78 s, total: 19.3 s
Wall time: 40.4 s
[{'weight': [0.31351351351351353, 0.0, 0.0, 0.6864864864864865]}, 0.9521847110667657, array([0.95

[0.3140495867768595, 0.008264462809917356, 0.0, 0.6776859504132231]
[0.3140495867768595, 0.0, 0.008264462809917356, 0.6776859504132231]
[0.3140495867768595, 0.0, 0.0, 0.6859504132231405]
[0.319672131147541, 0.0, 0.0, 0.680327868852459]
[0.3114754098360656, 0.00819672131147541, 0.0, 0.680327868852459]
[0.3114754098360656, 0.0, 0.0, 0.6885245901639344]
[0.3170731707317073, 0.0, 0.0, 0.6829268292682927]
[0.3089430894308943, 0.0, 0.008130081300813009, 0.6829268292682927]
[0.3089430894308943, 0.0, 0.0, 0.6910569105691057]
[0.31451612903225806, 0.0, 0.0, 0.6854838709677419]
[0.3064516129032258, 0.0, 0.008064516129032258, 0.6854838709677419]
[0.3064516129032258, 0.0, 0.0, 0.6935483870967742]
[0.312, 0.0, 0.0, 0.688]
[0.304, 0.0, 0.0, 0.696]
[0.30158730158730157, 0.007936507936507936, 0.0, 0.6904761904761905]
[0.30158730158730157, 0.0, 0.007936507936507936, 0.6904761904761905]
[0.30158730158730157, 0.0, 0.0, 0.6984126984126984]
[0.30708661417322836, 0.0, 0.0, 0.6929133858267716]
[0.29921259842

[0.3114754098360656, 0.0, 0.0, 0.6885245901639344]
[0.3089430894308943, 0.008130081300813009, 0.0, 0.6829268292682927]
[0.3089430894308943, 0.0, 0.008130081300813009, 0.6829268292682927]
[0.31451612903225806, 0.0, 0.0, 0.6854838709677419]
[0.3064516129032258, 0.008064516129032258, 0.0, 0.6854838709677419]
[0.3064516129032258, 0.0, 0.0, 0.6935483870967742]
[0.312, 0.0, 0.0, 0.688]
[0.304, 0.0, 0.008, 0.688]
[0.304, 0.0, 0.0, 0.696]
[0.30952380952380953, 0.0, 0.0, 0.6904761904761905]
[0.30158730158730157, 0.0, 0.007936507936507936, 0.6904761904761905]
[0.30708661417322836, 0.0, 0.0, 0.6929133858267716]
[0.2992125984251969, 0.0, 0.007874015748031496, 0.6929133858267716]
[0.3046875, 0.0, 0.0, 0.6953125]
[0.296875, 0.0078125, 0.0, 0.6953125]
[0.296875, 0.0, 0.0, 0.703125]
[0.29457364341085274, 0.007751937984496124, 0.0, 0.6976744186046512]
[0.29457364341085274, 0.0, 0.007751937984496124, 0.6976744186046512]
[0.3, 0.0, 0.0, 0.7]
[0.2923076923076923, 0.0, 0.007692307692307693, 0.7]
[0.3053435

[0.3140495867768595, 0.008264462809917356, 0.0, 0.6776859504132231]
[0.3140495867768595, 0.0, 0.0, 0.6859504132231405]
[0.3114754098360656, 0.00819672131147541, 0.0, 0.680327868852459]
[0.3114754098360656, 0.0, 0.0, 0.6885245901639344]
[0.3089430894308943, 0.008130081300813009, 0.0, 0.6829268292682927]
[0.3089430894308943, 0.0, 0.0, 0.6910569105691057]
[0.3064516129032258, 0.008064516129032258, 0.0, 0.6854838709677419]
[0.3064516129032258, 0.0, 0.008064516129032258, 0.6854838709677419]
[0.3064516129032258, 0.0, 0.0, 0.6935483870967742]
[0.304, 0.008, 0.0, 0.688]
[0.304, 0.0, 0.008, 0.688]
[0.304, 0.0, 0.0, 0.696]
[0.30952380952380953, 0.0, 0.0, 0.6904761904761905]
[0.30158730158730157, 0.0, 0.007936507936507936, 0.6904761904761905]
[0.30708661417322836, 0.0, 0.0, 0.6929133858267716]
[0.2992125984251969, 0.0, 0.007874015748031496, 0.6929133858267716]
[0.2992125984251969, 0.0, 0.0, 0.7007874015748031]
[0.3046875, 0.0, 0.0, 0.6953125]
[0.296875, 0.0, 0.0078125, 0.6953125]
[0.2945736434108

In [157]:
polls

[62, 0, 0, 138]